In [115]:
# include useful folders
import sys

import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

sys.path.append("../vendors/mtl_girnet/data_prep/")

import json
import h5py
import numpy as np
import glob
import random
import pandas as pd
import re
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt

# nltk
import nltk

# tokenizer
from twokenize import tokenizeRawTweetText as tokenize

# for a particular dataset
from xml.dom import minidom

In [116]:

### SemEval 2017 Task A

df = pd.read_csv("../data/datastories-semeval2017-task4/dataset/Subtask_A/4A-English/SemEval2017-task4-dev.subtask-A.english.INPUT.txt", sep="\t", header=None)

decode_map = {"negative": -1, "neutral": 0, "positive": 1}

df[1] = df[1].apply(lambda x: decode_map[x])
df[2] = df[2].apply(lambda x: tokenize(x))

data = map( lambda x :{'sentiment': x[1] , 'tokens': x[2] , 'text': ' '.join(x[2])} , df.to_numpy() )

en_semeval_17 = list(data)


### English-Spanish Code Mixed Data 

sents = {"N":-1 , "P" :1 , "NONE":0}

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_train.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_train = list(data)

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_test.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_test = list(data)

en_es_wssa_data = list(en_es_wssa_data_train) + list(en_es_wssa_data_test)

### Spanish Tweet Dataset

xmldoc = minidom.parse("../vendors/mtl_girnet/data_prep/data_cm_senti/general-tweets-train-tagged.xml")
tweets = xmldoc.getElementsByTagName('tweet')

sents = {"N":-1 , "P" :1 , "NEU":0 , 'NONE':0 , "P+" : 1 , "N+":-1 }


es_tass1_data = []

for i in range( len(tweets)-1) :
    if i == 6055:
        continue # bad jogar
    textt = tweets[i].getElementsByTagName('content')[0].childNodes[0].data
    words = tokenize( textt )
    sentiment = tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('value')[0].childNodes[0].data
    assert len(tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('entity'))==0
    es_tass1_data.append({'text':textt , 'tokens':words , 'sentiment': sents[sentiment] })

### Some english tweet data

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/twitter4242.txt", "r", encoding="utf-8",errors='ignore').read().split("\n")[1:-1]
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

en_twitter_data = list(data)

### es2_twitter_data

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_dev_complete.txt", encoding="utf-8").read().split("\n")[1:-1]
data += open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_test_average_complete.tsv", encoding="utf-8").read().split("\n")[1:-2]

data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

es2_twitter_data = list(data)

def get_y(data):
    from keras.utils import to_categorical
    y = []
    for row in data:
        y.append(int(row['sentiment']))
    y = to_categorical(y,num_classes=3)
    return y


print("Code-Mixed: en_es_wssa_data: %d" % len(en_es_wssa_data))
print("Spanish: es2_twitter_data: %d" % len(es2_twitter_data))
print("Spanish: es_tass1_data: %d" % len(es_tass1_data))
print("English: en_twitter_data: %d" % len(en_twitter_data))
# print("English: en_sentiment140: %d" %len(en_sentiment140))
en_es_y =  get_y(en_es_wssa_data)
en_es_y_train =  get_y(en_es_wssa_data_train)
en_es_y_test =  get_y(en_es_wssa_data_test)
es_twitter_y = get_y(es2_twitter_data)
es_tass_y = get_y(es_tass1_data)
en_twitter_y = get_y(en_twitter_data)
en_semeval_17_y = get_y(en_semeval_17)
# en_sentiment140_y = get_y(en_sentiment140)

Code-Mixed: en_es_wssa_data: 3062
Spanish: es2_twitter_data: 3202
Spanish: es_tass1_data: 7217
English: en_twitter_data: 4241


In [117]:
from keras import backend as K


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
def get_class_weight(y):
    """
    Used from: https://stackoverflow.com/a/50695814
    TODO: check validity and 'balanced' option
    :param y: A list of one-hot-encoding labels [[0,0,1,0],[0,0,0,1],..]
    :return: class-weights to be used by keras model.fit(.. class_weight="") -> {0:0.52134, 1:1.adas..}
    """
    y_integers = np.argmax(y, axis=1)
    class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
    d_class_weights = dict(enumerate(class_weights))
    return d_class_weights

In [119]:
# ! pip install bpemb

In [1]:
from bpemb import BPEmb
multibpemb = BPEmb(lang="multi", vs=1000000, dim=300)

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [ ]:
multibpemb.

In [125]:
multibpemb.embed("sup dfsdg").shape

(3, 300)

In [127]:
max_len = 32
zero_vector = [0 for _ in range(300)]
def get_x(data_):
    x_  = []
    for sent in data_:
        pred = list(multibpemb.embed(sent['text']))
        if len(pred) >= 32:
            pred = pred[:32]
        else:
            counter = len(pred)
            while counter < max_len:
                pred.append(zero_vector)
                counter = counter + 1
        x_.append(pred)
    return np.array(x_)
en_es_x =  get_x(en_es_wssa_data)
es_twitter_x = get_x(es2_twitter_data)
es_tass_x = get_x(es_tass1_data)
en_twitter_x = get_x(en_twitter_data)
en_semeval_17_x = get_x(en_semeval_17)
en_es_x_train =  get_x(en_es_wssa_data_train)
en_es_x_test =  get_x(en_es_wssa_data_test)

In [26]:
import fasttext

In [27]:
embed = fasttext.load_model('../vendors/language-models/all_p_fasttext.bin')

In [101]:
max_len = 32
zero_vector = [0 for _ in range(100)]
def get_x(data_):
#     x_  = []
#     for sent in data_:
#         x_.append(embed.get_sentence_vector(sent['text'].replace("\n"," ")))
#     return np.array(x_)
    x_  = []
    for sent in data_:
        tokenised = fasttext.tokenize(sent['text'])
        sent_vector = []
        counter = 0
        for token in tokenised:
            if counter >= max_len:
                break
            else:
                sent_vector.append(embed[token])
                counter = counter + 1
        
        if counter < max_len:
            sent_vector.append(embed['</s>'])
            counter = counter + 1
                               
        while counter < max_len:
            sent_vector.append(zero_vector)
            counter = counter + 1
            
        x_.append(sent_vector)
        
    return np.array(x_)

en_es_x =  get_x(en_es_wssa_data)
es_twitter_x = get_x(es2_twitter_data)
es_tass_x = get_x(es_tass1_data)
en_twitter_x = get_x(en_twitter_data)
en_semeval_17_x = get_x(en_semeval_17)
en_es_x_train =  get_x(en_es_wssa_data_train)
en_es_x_test =  get_x(en_es_wssa_data_test)

In [129]:
from keras.layers import *
from keras.models import Sequential
from keras.preprocessing import sequence

In [38]:
model = Sequential()
model.add(Dense(50, input_shape=(100,)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy', f1])

In [130]:
model = Sequential()
model.add(LSTM(150, dropout=0.3, input_shape=(32, 300), recurrent_dropout=0.3))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1])

In [131]:
history = model.fit(en_twitter_x, en_twitter_y, epochs=10, shuffle=True, validation_split=0.2)

Train on 3392 samples, validate on 849 samples
Epoch 1/10
3392/3392 [==============================] - 11s 3ms/step - loss: 1.0226 - acc: 0.4634 - f1: 0.3069 - val_loss: 0.9533 - val_acc: 0.5159 - val_f1: 0.4085
Epoch 2/10
3392/3392 [==============================] - 7s 2ms/step - loss: 0.9663 - acc: 0.5150 - f1: 0.3896 - val_loss: 0.9205 - val_acc: 0.5300 - val_f1: 0.4216
Epoch 3/10
3392/3392 [==============================] - 7s 2ms/step - loss: 0.9358 - acc: 0.5501 - f1: 0.4251 - val_loss: 0.8772 - val_acc: 0.5665 - val_f1: 0.5142
Epoch 4/10
3392/3392 [==============================] - 7s 2ms/step - loss: 0.9070 - acc: 0.5699 - f1: 0.4694 - val_loss: 0.9140 - val_acc: 0.5147 - val_f1: 0.4526
Epoch 5/10
3392/3392 [==============================] - 6s 2ms/step - loss: 0.8769 - acc: 0.5846 - f1: 0.5054 - val_loss: 0.8557 - val_acc: 0.5901 - val_f1: 0.4936
Epoch 6/10
3392/3392 [==============================] - 4s 1ms/step - loss: 0.8498 - acc: 0.6212 - f1: 0.5494 - val_loss: 0.8406 - v

In [132]:
history = model.fit(en_semeval_17_x, en_semeval_17_y, epochs=10, validation_split=0.2, shuffle=True)

Train on 16505 samples, validate on 4127 samples
Epoch 1/10
16505/16505 [==============================] - 26s 2ms/step - loss: 0.8814 - acc: 0.5792 - f1: 0.5272 - val_loss: 0.8772 - val_acc: 0.5820 - val_f1: 0.5580
Epoch 2/10
16505/16505 [==============================] - 25s 2ms/step - loss: 0.7994 - acc: 0.6330 - f1: 0.6040 - val_loss: 0.8523 - val_acc: 0.6021 - val_f1: 0.5708
Epoch 3/10
16505/16505 [==============================] - 23s 1ms/step - loss: 0.7534 - acc: 0.6571 - f1: 0.6355 - val_loss: 0.8499 - val_acc: 0.6104 - val_f1: 0.5929
Epoch 4/10
16505/16505 [==============================] - 25s 1ms/step - loss: 0.7164 - acc: 0.6771 - f1: 0.6593 - val_loss: 0.8350 - val_acc: 0.6198 - val_f1: 0.6032
Epoch 5/10
16505/16505 [==============================] - 24s 1ms/step - loss: 0.6822 - acc: 0.6958 - f1: 0.6812 - val_loss: 0.8463 - val_acc: 0.6230 - val_f1: 0.6155
Epoch 6/10
16505/16505 [==============================] - 25s 2ms/step - loss: 0.6570 - acc: 0.7135 - f1: 0.7014 - v

In [133]:
history = model.fit(es_tass_x, es_tass_y, epochs=10, validation_split=0.2, shuffle=True)

Train on 5773 samples, validate on 1444 samples
Epoch 1/10
5773/5773 [==============================] - 9s 2ms/step - loss: 1.0503 - acc: 0.4880 - f1: 0.3725 - val_loss: 0.8997 - val_acc: 0.5824 - val_f1: 0.5117
Epoch 2/10
5773/5773 [==============================] - 8s 1ms/step - loss: 0.9132 - acc: 0.5694 - f1: 0.5010 - val_loss: 0.8611 - val_acc: 0.6143 - val_f1: 0.5695
Epoch 3/10
5773/5773 [==============================] - 8s 1ms/step - loss: 0.8548 - acc: 0.6097 - f1: 0.5590 - val_loss: 0.8483 - val_acc: 0.6039 - val_f1: 0.5570
Epoch 4/10
5773/5773 [==============================] - 7s 1ms/step - loss: 0.8141 - acc: 0.6364 - f1: 0.5966 - val_loss: 0.8355 - val_acc: 0.6080 - val_f1: 0.5815
Epoch 5/10
5773/5773 [==============================] - 8s 1ms/step - loss: 0.7671 - acc: 0.6669 - f1: 0.6344 - val_loss: 0.8596 - val_acc: 0.5976 - val_f1: 0.5558
Epoch 6/10
5773/5773 [==============================] - 8s 1ms/step - loss: 0.7249 - acc: 0.6834 - f1: 0.6553 - val_loss: 0.8776 - v

In [134]:
history = model.fit(es_twitter_x, es_twitter_y, epochs=20, validation_split=0.2, shuffle=True)

Train on 2561 samples, validate on 641 samples
Epoch 1/20
2561/2561 [==============================] - 3s 1ms/step - loss: 1.0611 - acc: 0.4924 - f1: 0.3907 - val_loss: 0.9785 - val_acc: 0.4992 - val_f1: 0.4193
Epoch 2/20
2561/2561 [==============================] - 4s 2ms/step - loss: 0.9426 - acc: 0.5463 - f1: 0.4559 - val_loss: 0.9770 - val_acc: 0.5257 - val_f1: 0.4598
Epoch 3/20
2561/2561 [==============================] - 3s 1ms/step - loss: 0.8832 - acc: 0.5955 - f1: 0.5190 - val_loss: 0.9660 - val_acc: 0.5460 - val_f1: 0.4689
Epoch 4/20
2561/2561 [==============================] - 4s 2ms/step - loss: 0.8344 - acc: 0.6111 - f1: 0.5762 - val_loss: 0.9691 - val_acc: 0.5382 - val_f1: 0.4854
Epoch 5/20
2561/2561 [==============================] - 4s 2ms/step - loss: 0.7999 - acc: 0.6361 - f1: 0.5937 - val_loss: 0.9701 - val_acc: 0.5632 - val_f1: 0.5004
Epoch 6/20
2561/2561 [==============================] - 4s 1ms/step - loss: 0.7407 - acc: 0.6747 - f1: 0.6353 - val_loss: 1.0348 - va

In [135]:
x = np.concatenate([en_semeval_17_x, en_twitter_x, es_tass_x, es_twitter_x])
y = np.concatenate([en_semeval_17_y, en_twitter_y, es_tass_y, es_twitter_y])

In [136]:
history = model.fit(x, y, epochs=30, validation_data=(en_es_x_train,en_es_y_train), shuffle=True)

Train on 35292 samples, validate on 2449 samples
Epoch 1/30
35292/35292 [==============================] - 49s 1ms/step - loss: 0.7408 - acc: 0.6751 - f1: 0.6570 - val_loss: 1.0034 - val_acc: 0.5223 - val_f1: 0.4753
Epoch 2/30
35292/35292 [==============================] - 51s 1ms/step - loss: 0.6705 - acc: 0.7101 - f1: 0.6956 - val_loss: 0.9707 - val_acc: 0.5459 - val_f1: 0.5126
Epoch 3/30
35292/35292 [==============================] - 50s 1ms/step - loss: 0.6380 - acc: 0.7253 - f1: 0.7150 - val_loss: 0.9990 - val_acc: 0.5382 - val_f1: 0.4958
Epoch 4/30
35292/35292 [==============================] - 50s 1ms/step - loss: 0.6148 - acc: 0.7355 - f1: 0.7274 - val_loss: 0.9805 - val_acc: 0.5508 - val_f1: 0.5188
Epoch 5/30
35292/35292 [==============================] - 50s 1ms/step - loss: 0.5946 - acc: 0.7476 - f1: 0.7384 - val_loss: 1.0192 - val_acc: 0.5308 - val_f1: 0.4908
Epoch 6/30
35292/35292 [==============================] - 50s 1ms/step - loss: 0.5749 - acc: 0.7533 - f1: 0.7466 - v

KeyboardInterrupt: 

In [137]:
model.evaluate(en_es_x_test, en_es_y_test)

613/613 [==============================] - 0s 497us/step


[1.1475409948028514, 0.5285481240290413, 0.5210127912182209]

In [138]:
history = model.fit(en_es_x_train, en_es_y_train, epochs=20, validation_data=(en_es_x_test, en_es_y_test), shuffle=True)

Train on 2449 samples, validate on 613 samples
Epoch 1/20
2449/2449 [==============================] - 4s 2ms/step - loss: 1.0959 - acc: 0.5394 - f1: 0.5123 - val_loss: 0.9578 - val_acc: 0.5889 - val_f1: 0.5593
Epoch 2/20
2449/2449 [==============================] - 4s 1ms/step - loss: 0.9163 - acc: 0.5827 - f1: 0.5598 - val_loss: 0.9329 - val_acc: 0.6020 - val_f1: 0.5732
Epoch 3/20
2449/2449 [==============================] - 4s 2ms/step - loss: 0.8105 - acc: 0.6386 - f1: 0.6131 - val_loss: 0.9105 - val_acc: 0.6199 - val_f1: 0.5896
Epoch 4/20
2449/2449 [==============================] - 4s 2ms/step - loss: 0.7679 - acc: 0.6497 - f1: 0.6273 - val_loss: 0.9038 - val_acc: 0.6248 - val_f1: 0.6088
Epoch 5/20
2449/2449 [==============================] - 3s 1ms/step - loss: 0.6960 - acc: 0.7011 - f1: 0.6864 - val_loss: 0.8926 - val_acc: 0.6362 - val_f1: 0.6115
Epoch 6/20
2449/2449 [==============================] - 4s 2ms/step - loss: 0.6447 - acc: 0.7338 - f1: 0.7198 - val_loss: 0.9099 - va

In [139]:
model.evaluate(en_es_x_test, en_es_y_test)

613/613 [==============================] - 0s 347us/step


[1.1387531876952963, 0.6362153345181153, 0.6377271510065088]